In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

%load_ext autoreload
%autoreload 2

/home/ylu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import SPACEL
from SPACEL import Scube, Splane

In [3]:
data_folder = "../../data/STARMapPlus/"
method = 'SPACEL'
i = 3
adata = ad.read_h5ad(os.path.join(data_folder, f"sagittal{i}.h5ad"))
scale = np.linalg.norm(adata.obsm['spatial'].max(0) - adata.obsm['spatial'].min(0))
slices = split_slice(
    adata=adata,
    spatial_key='spatial_raw',
    split_num=2,
    axis=2,
)

In [4]:
slices[0]

AnnData object with n_obs × n_vars = 103795 × 1022
    obs: 'Sub_molecular_cell_type', 'Main_molecular_tissue_region', 'Sub_molecular_tissue_region', 'Molecular_spatial_cell_type', 'slice'
    obsm: 'spatial', 'spatial_raw'

In [5]:
overlay_ratio_set = np.arange(0.5, 1.05, 0.05)
from tqdm import tqdm
maes = []
for overlay_ratio in tqdm(overlay_ratio_set):
    slice1 = slices[0].copy()
    slice2 = slices[1].copy()
    slice1_crop, slice2_crop = crop_slices(
        slice1, 
        slice2, 
        spatial_key='spatial', 
        overlay_ratio = overlay_ratio,
    )
    slice1, slice2 = slice1_crop.copy(), slice2_crop.copy()
    sampling_num = 20000
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    spatial_key = 'spatial'
    key_added = 'align_spatial'
    slice1.obsm['spatial_2D'] = slice1.obsm[spatial_key]
    slice2.obsm['spatial_2D'] = slice2.obsm[spatial_key]
    slice1.obsm['spatial'] = slice1.obsm[spatial_key]
    slice2.obsm['spatial'] = slice2.obsm[spatial_key]
    align_slices = [slice1, slice2]
    Scube.align(
        align_slices,
        cluster_key='Main_molecular_tissue_region', 
        n_neighbors = 15, 
        n_threads=10,
        p=2,
        write_loc_path=os.path.join('results', 'Scube_outputs/aligned_coordinates.csv')
    )
    align_slices[0].obsm[key_added] = align_slices[0].obsm['spatial_aligned'].values
    align_slices[1].obsm[key_added] = align_slices[1].obsm['spatial_aligned'].values
    R1, t1=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    R2, t2=solve_RT_by_correspondence(align_slices[0].obsm[spatial_key], align_slices[0].obsm[key_added])
    t = t1 @ R2.T + t2
    R = R2 @ R1
    
    spatial_align = align_slices[1].obsm['spatial'] @ R.T + t
    spatial_GT = align_slices[1].obsm['spatial']
    mae, mae_var = MAE(spatial_align, spatial_GT) / scale
    maes.append(mae)
np.save(f'./results/{method}_different_overlap.npy', {'maes': maes, 'overlay_ratio':overlay_ratio}, allow_pickle=True)

  0%|                                      | 0/11 [00:00<?, ?it/s]

Start alignment...
Alignment slice 1 to 0
Runtime: 213.501606464386 s


  9%|██▋                          | 1/11 [03:34<35:43, 214.31s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 208.87518191337585 s


 18%|█████▎                       | 2/11 [07:03<31:43, 211.54s/it]

...

 82%|███████████████████████▋     | 9/11 [33:11<07:15, 217.99s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 215.32881498336792 s


 91%|█████████████████████████▍  | 10/11 [36:48<03:37, 217.43s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 236.20528650283813 s


100%|████████████████████████████| 11/11 [40:45<00:00, 222.29s/it]
